In [ ]:
%pip install datasets --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from datasets import load_dataset
import os
import re

# Download the dataset
dataset = load_dataset("princeton-nlp/SWE-bench_Lite")

# Convert the dataset to a Pandas DataFrame
df = pd.DataFrame(dataset["dev"])

# Print the DataFrame
print(df.head())

/Users/berrazuriz/Desktop/Blar/repositories/code-base-agent/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                repo              instance_id  \
0  sqlfluff/sqlfluff  sqlfluff__sqlfluff-1625   
1  sqlfluff/sqlfluff  sqlfluff__sqlfluff-2419   
2  sqlfluff/sqlfluff  sqlfluff__sqlfluff-1733   
3  sqlfluff/sqlfluff  sqlfluff__sqlfluff-1517   
4  sqlfluff/sqlfluff  sqlfluff__sqlfluff-1763   

                                base_commit  \
0  14e1a23a3166b9a645a16de96f694c77a5d4abb7   
1  f1dba0e1dd764ae72d67c3d5e1471cf14d3db030   
2  a1579a16b1d8913d9d7c7d12add374a290bcc78c   
3  304a197829f98e7425a46d872ada73176137e5ae   
4  a10057635e5b2559293a676486f0b730981f037a   

                                               patch  \
0  diff --git a/src/sqlfluff/rules/L031.py b/src/...   
1  diff --git a/src/sqlfluff/rules/L060.py b/src/...   
2  diff --git a/src/sqlfluff/rules/L039.py b/src/...   
3  diff --git a/src/sqlfluff/core/parser/helpers....   
4  diff --git a/src/sqlfluff/core/linter/linted_f...   

                                          test_patch  \
0  diff --git a/test/cli/comm

In [ ]:
# Define a function to extract file names from git diff output
def extract_filenames(patch):
    pattern = r"diff --git a/(.*?) b/(.*?)\n"
    matches = re.findall(pattern, patch)
    # Return a list of unique file names
    return list(set([match[0] for match in matches]))


# Apply the function to the 'patch' column
df["edited_files"] = df["patch"].apply(extract_filenames)

In [ ]:
df_first_row = df.iloc[0:1]

In [ ]:
downloaded_repos = {}

,repo,instance_id,base_commit,patch,test_patch,problem_statement,hints_text,created_at,version,FAIL_TO_PASS,PASS_TO_PASS,environment_setup_commit,edited_files
0,sqlfluff/sqlfluff,sqlfluff__sqlfluff-1625,14e1a23a3166b9a645a16de96f694c77a5d4abb7,diff --git a/src/sqlfluff/rules/L031.py b/src/...,diff --git a/test/cli/commands_test.py b/test/...,"TSQL - L031 incorrectly triggers ""Avoid using ...","Actually, re-reading the docs I think this is ...",2021-10-13T11:35:29Z,0.6,"[""test/cli/commands_test.py::test__cli__comman...","[""test/cli/commands_test.py::test__cli__comman...",67023b85c41d23d6c6d69812a41b207c4f8a9331,[src/sqlfluff/rules/L031.py]


In [ ]:
import uuid
from blar_graph.graph_construction.core.graph_builder import GraphConstructor
from blar_graph.db_managers import Neo4jManager
from blar_graph.agents_tools.agents_examples.metamate import get_metamate_agent
import subprocess

try:
    os.makedirs("repos", exist_ok=True)  # exist_ok=True avoids error if directory already exists
    print("Directory 'repos' ensured.")
except Exception as e:
    print(f"An error occurred ensuring the directory: {e}")

for index, row in df_first_row.iterrows():
    repo = row["repo"]
    print(repo)
    if downloaded_repos.get(repo) is None:
        repo_path = f"repos/{repo}"
        if not os.path.exists(f"repos/{repo}"):
            os.makedirs(repo_path, exist_ok=True)
            subprocess.run(f"git clone https://github.com/{repo}.git {repo_path}", shell=True, check=True)
            subprocess.run(f"git checkout {row['base_commit']}", shell=True, check=True)
            subprocess.run(f"find {repo_path} -type l -delete", shell=True, check=True)
        repo_id = str(uuid.uuid4())
        graph_manager = Neo4jManager(repo_id)
        graph_constructor = GraphConstructor(graph_manager, "python")
        graph_constructor.build_graph(f"repos/{repo}")
        downloaded_repos[repo] = graph_manager

    graph_manager = downloaded_repos[repo]
    agent = get_metamate_agent(graph_manager)
    list(agent.stream({"input": f"Proble: {row['problem_statement']}\nHints: {row['hints_text']}"}))

Directory 'repos' ensured.
sqlfluff/sqlfluff
Processed repos/sqlfluff/sqlfluff/plugins/sqlfluff-templater-dbt/sqlfluff_templater_dbt/__init__.py
Processed repos/sqlfluff/sqlfluff/plugins/sqlfluff-templater-dbt/sqlfluff_templater_dbt/templater.py
Processed repos/sqlfluff/sqlfluff/plugins/sqlfluff-templater-dbt/setup.py
Processed repos/sqlfluff/sqlfluff/plugins/sqlfluff-plugin-example/setup.py
Processed repos/sqlfluff/sqlfluff/plugins/sqlfluff-plugin-example/src/sqlfluff_plugin_example/__init__.py
Processed repos/sqlfluff/sqlfluff/plugins/sqlfluff-plugin-example/src/sqlfluff_plugin_example/rules.py
Processed repos/sqlfluff/sqlfluff/util.py
Processed repos/sqlfluff/sqlfluff/docs/source/conf.py
Processed repos/sqlfluff/sqlfluff/docs/source/_ext/sqlfluff_domain.py
Processed repos/sqlfluff/sqlfluff/docs/generate-rule-docs.py
Processed repos/sqlfluff/sqlfluff/examples/01_basic_api_usage.py
Processed repos/sqlfluff/sqlfluff/examples/04_config_overrides.py
Processed repos/sqlfluff/sqlfluff/exam

KeyboardInterrupt: 

In [ ]:
for index, row in df_first_row.iterrows():
    graph_manager = Neo4jManager("42b69d2a-d960-465e-992a-c51b988ae375")
    agent = get_metamate_agent(graph_manager)
    list(agent.stream({"input": f"Problem: {row['problem_statement']}\nHints: {row['hints_text']}"}))



> Entering new AgentExecutor chain...

Invoking: `keword_search` with `{'query': 'TSQL L031'}`


current node code:
 """The MSSQL T-SQL dialect.

https://docs.microsoft.com/en-us/sql/t-sql/language-elements/language-elements-transact-sql
"""

from sqlfluff.core.dialects import load_raw_dialect
from sqlfluff.core.parser import (
    AnyNumberOf,
    AnySetOf,
    BaseFileSegment,
    BaseSegment,
    Bracketed,
    CodeSegment,
    CommentSegment,
    CompositeBinaryOperatorSegment,
    CompositeComparisonOperatorSegment,
    Conditional,
    Dedent,
    Delimited,
    IdentifierSegment,
    ImplicitIndent,
    Indent,
    LiteralSegment,
    Matchable,
    MultiStringParser,
    NewlineSegment,
    Nothing,
    OneOf,
    OptionallyBracketed,
    ParseMode,
    Ref,
    RegexLexer,
    RegexParser,
    SegmentGenerator,
    Sequence,
    TypedParser,
    WhitespaceSegment,
    WordSegment,
)
from sqlfluff.dialects import dialect_ansi as ansi
from sqlfluff.dialects.dialect_tsql_keywor

In [ ]:
downloaded_repos

{'sqlfluff/sqlfluff': <blar_graph.db_managers.neo4j_manager.Neo4jManager at 0x1099b7e20>}